In [41]:
%load_ext dotenv
%dotenv

In [42]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import psycopg2
from sqlalchemy import create_engine
import datetime
import os

In [43]:
database_url = os.getenv('DATABASE_URL')
engine = create_engine(database_url)

In [44]:
url = 'https://datos.gob.ar/dataset/transporte-sube---cantidad-transacciones-usos-por-fecha'

In [45]:
def agregar_anio_mes(row):
    fecha = row['DIA_TRANSPORTE']
    anio, mes, _ = fecha.split("-")
    periodo = anio+"-"+mes
    return anio, mes, periodo

In [46]:
fecha_hora_actual = datetime.datetime.now()
fecha_hora_formateada = fecha_hora_actual.strftime("%Y-%m-%d %H:%M:%S")
print("Fecha y hora actual:", fecha_hora_formateada)

Fecha y hora actual: 2023-10-28 12:18:44


In [47]:
response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.select(".pkg-container .pkg-actions a")
    for link in links:
        href = link.get('href')
        if href and '/dataset/transporte' not in href:
            print(href)
            df = pd.read_csv(href)
            df['Anio'], df['Mes'], df['Periodo'] = zip(*df.apply(agregar_anio_mes, axis=1))
            df.rename(columns={
                'DIA_TRANSPORTE': 'Fecha Viaje',
                'NOMBRE_EMPRESA': 'Nombre Empresa',
                'LINEA': 'Linea',
                'AMBA': 'Amba',
                'TIPO_TRANSPORTE': 'Tipo Transporte',
                'JURISDICCION': 'Jurisdiccion',
                'PROVINCIA': 'Provincia',
                'MUNICIPIO': 'Municipio',
                'CANTIDAD': 'Cantidad',
                'DATO_PRELIMINAR': 'Dato Preliminar'
            }, inplace=True)
            anio = href[-8:-4]
            df.to_sql(f'Transacciones_SUBE_{anio}', engine, if_exists='replace', index=False)
            print(f'Tabla Transacciones_SUBE_{anio} subida correctamente')
else:
    print("This page could not be accessed::", response.status_code)

https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2023.csv
Tabla Transacciones_SUBE_2023 subida correctamente
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2022.csv
Tabla Transacciones_SUBE_2022 subida correctamente
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2021.csv
Tabla Transacciones_SUBE_2021 subida correctamente
https://archivos-datos.transporte.gob.ar/upload/Dat_Ab_Usos/dat-ab-usos-2020.csv
Tabla Transacciones_SUBE_2020 subida correctamente


In [48]:
fecha_hora_actual = datetime.datetime.now()
fecha_hora_formateada = fecha_hora_actual.strftime("%Y-%m-%d %H:%M:%S")
print("Fecha y hora actual:", fecha_hora_formateada)

Fecha y hora actual: 2023-10-28 12:31:20
